# Section 1: Downloading Flight for Testing datasets

This section downloads both departure and arrival dataset from [tablebuilder.singsat](https://tablebuilder.singstat.gov.sg/), Depart of Statistics Singapore website. Files are dowloaded in our google drive and extract from there in the code. 2023-2024 will represent 30%.

In [ ]:
#To download from google drive
pip install gdown

## Section 1.1 : Downloading Flight Departure For Testing

In [ ]:
import gdown
import pandas as pd

# Google Drive Departure Dataset 23/24 ID
fileDep_id = "1CfEgn8RMfwpG_0RyUc3RvQFf03_LWQRQ"
fileDep_name = "departure_test-23-24.csv"

# Download the file
gdown.download(f"https://drive.google.com/uc?id={fileDep_id}", fileDep_name, quiet=False)

# Load the CSV file into a Pandas DataFrame
df_d_test = pd.read_csv(fileDep_name, header=9)

# Remove NaN entries
df_d_test = df_d_test.dropna()

df_d_test.set_index(df_d_test.columns[0], inplace=True)

# Transpose the DataFrame (convert months → columns, countries → rows)
df_d_test = df_d_test.T.reset_index()

# Rename columns
df_d_test.rename(columns={"index": "Country"}, inplace=True)

# Reshape using melt()
df_long_test = df_d_test.melt(id_vars=["Country"], var_name="Month (YYYY-MM)", value_name="Departures")

# Save or display results
#df_long.to_csv("departures_refactor.csv", index=False)
print(df_long_test.head())  # View first few rows


In [ ]:

pd.set_option('display.max_colwidth', None)  # Show full content of each column
pd.set_option('display.max_rows', None)      # Show all rows
pd.set_option('display.max_columns', None)

# Remove the common prefix and suffix from the 'Country' column
df_long_test['Country'] = df_long_test['Country'].str.replace('Number Of Air Passenger Departures -> ', '', regex=False)
df_long_test['Country'] = df_long_test['Country'].str.replace(' (Number)', '', regex=False)

# Display the updated DataFrame
print(df_long_test.head(30))

In [10]:
# Create a new DataFrame with only country names
df_countries_test = df_long_test[df_long_test['Country'].str.contains('->')].copy()

# Extract the country names by splitting and taking the last part
df_countries_test['Country'] = df_countries_test['Country'].apply(lambda x: x.split('->')[-1].strip())

# Rest index numbering for each entry
df_countries_index_test = df_countries_test.reset_index(drop=True)

# Change to interger type for Departure column
df_countries_index_test['Departures'] = df_countries_index_test['Departures'].astype(int)

df_countries_index_test['Month (YYYY-MM)'] = pd.to_datetime(
    df_countries_index_test['Month (YYYY-MM)'].str.strip(), format='%Y %b'
).dt.strftime('%Y-%m')


# Export into CSV
df_countries_index_test.to_csv("departures_test_final.csv", index=False)

# Display the new DataFrame
df_countries_index_dep_test = df_countries_index_test




## Section 1.2 : Downloading Flight Arrivals For Testing

In [ ]:
import gdown
import pandas as pd

# Google Drive Arrival Dataset 23/24 ID
fileArr_id = "10KuTnelehV6N54pZos56sXkVH7Z5HIfY"
fileArr_name = "arrival_test-23-24.csv"


# Download the file
gdown.download(f"https://drive.google.com/uc?id={fileArr_id}", fileArr_name, quiet=False)

# Load the CSV file into a Pandas DataFrame
df_a_test = pd.read_csv(fileArr_name, header=9)

# Remove NaN entries
df_a_test = df_a_test.dropna()

df_a_test.set_index(df_a_test.columns[0], inplace=True)

# Transpose the DataFrame (convert months → columns, countries → rows)
df_a_test = df_a_test.T.reset_index()

# Rename columns
df_a_test.rename(columns={"index": "Country"}, inplace=True)

# Reshape using melt()
df_arr_test = df_a_test.melt(id_vars=["Country"], var_name="Month (YYYY-MM)", value_name="Arrivals")

# Save or display results
print(df_arr_test.head())  # View first few rows


In [ ]:

pd.set_option('display.max_colwidth', None)  # Show full content of each column
pd.set_option('display.max_rows', None)      # Show all rows
pd.set_option('display.max_columns', None)

# Remove the common prefix and suffix from the 'Country' column
df_arr_test['Country'] = df_arr_test['Country'].str.replace('Number Of Air Passenger Arrivals -> ', '', regex=False)
df_arr_test['Country'] = df_arr_test['Country'].str.replace(' (Number)', '', regex=False)

# Display the updated DataFrame
print(df_arr_test.head(30))

In [ ]:
# Create a new DataFrame with only country names
df_countries_arr_test = df_arr_test[df_arr_test['Country'].str.contains('->')].copy()

# Extract the country names by splitting and taking the last part
df_countries_arr_test['Country'] = df_countries_arr_test['Country'].apply(lambda x: x.split('->')[-1].strip())

# Rest index numbering for each entry
df_countries_index_arr_test = df_countries_arr_test.reset_index(drop=True)

# Change to interger type for Arrivals column
df_countries_index_arr_test['Arrivals'] = pd.to_numeric(df_countries_index_arr_test['Arrivals'], errors='coerce').fillna(0).astype(int)

# Changing format for date 
df_countries_index_arr_test['Month (YYYY-MM)'] = pd.to_datetime(
    df_countries_index_arr_test['Month (YYYY-MM)'].str.strip(), format='%Y %b'
).dt.strftime('%Y-%m')

# Export to CSV
df_countries_index_arr_test.to_csv("arrivals_test_final.csv", index=False)

# Display the new DataFrame
df_countries_index_arr_test

## Section 1.3 : Merging Flight Departures & Arrivals For Testing

In [14]:
import pandas as pd

# Merge based on 'Country' and 'Month (YYYY-MM)'
merged_df_test = pd.merge(df_countries_index_arr_test, df_countries_index_dep_test, on=["Country", "Month (YYYY-MM)"], how="inner")

merged_df_test.to_csv("merged_flight_testing.csv",index=False )


# Section 2: Downloading Flight Training datasets

This section downloads both departure and arrival dataset from [tablebuilder.singsat](https://tablebuilder.singstat.gov.sg/), Depart of Statistics Singapore website. Files are dowloaded in our google drive and extract from there in the code. Remaining 70% will be base of (24 months / 30%) * 70% = **56 months**. We will take 56 months before [covid-19](https://en.wikipedia.org/wiki/COVID-19_pandemic#:~:text=The%20COVID%2D19%20pandemic%20(also,then%20worldwide%20in%20early%202020.))(Dec 19) as it will be the outlier of our dataset.

## Section 2.1 : Downloading Flight Departure For Training

In [ ]:
import gdown
import pandas as pd

# Google Drive Departure Dataset 15/19 ID
fileDep_id = "12IQx7qGJGd3i1rUVUo5L1eaN7lGa9H16"
fileDep_name = "departure_train-15-24.csv"

# Download the file
gdown.download(f"https://drive.google.com/uc?id={fileDep_id}", fileDep_name, quiet=False)

# Load the CSV file into a Pandas DataFrame
df_d_train = pd.read_csv(fileDep_name, header=9)

# Remove NaN entries
df_d_train = df_d_train.dropna()

df_d_train.set_index(df_d_train.columns[0], inplace=True)

# Transpose the DataFrame (convert months → columns, countries → rows)
df_d_train = df_d_train.T.reset_index()

# Rename columns
df_d_train.rename(columns={"index": "Country"}, inplace=True)

# Reshape using melt()
df_long_train = df_d_train.melt(id_vars=["Country"], var_name="Month (YYYY-MM)", value_name="Departures")

# Save or display results
#df_long.to_csv("departures_refactor.csv", index=False)
print(df_long_train.head())  # View first few rows


In [ ]:

pd.set_option('display.max_colwidth', None)  # Show full content of each column
pd.set_option('display.max_rows', None)      # Show all rows
pd.set_option('display.max_columns', None)

# Remove the common prefix and suffix from the 'Country' column
df_long_train['Country'] = df_long_train['Country'].str.replace('Number Of Air Passenger Departures -> ', '', regex=False)
df_long_train['Country'] = df_long_train['Country'].str.replace(' (Number)', '', regex=False)

# Display the updated DataFrame
print(df_long_train.head(30))

In [17]:
# Create a new DataFrame with only country names
df_countries_train = df_long_train[df_long_train['Country'].str.contains('->')].copy()

# Extract the country names by splitting and taking the last part
df_countries_train['Country'] = df_countries_train['Country'].apply(lambda x: x.split('->')[-1].strip())

# Rest index numbering for each entry
df_countries_index_train = df_countries_train.reset_index(drop=True)

# Change to interger type for Departure column
df_countries_index_train['Departures'] = df_countries_index_train['Departures'].astype(int)

df_countries_index_train['Month (YYYY-MM)'] = pd.to_datetime(
    df_countries_index_train['Month (YYYY-MM)'].str.strip(), format='%Y %b'
).dt.strftime('%Y-%m')


# Export into CSV
df_countries_index_train.to_csv("departures_train_final.csv", index=False)

# Display the new DataFrame
df_countries_index_dep_train = df_countries_index_train




## Section 2.2 : Downloading Flight Arrivals For Training

In [ ]:
import gdown
import pandas as pd

# Google Drive Arrival Dataset 15/19 ID
fileArr_id = "1KayLfND03TnB7JNCERYAV1d6GaDPpN2L"
fileArr_name = "arrival_test-15-19.csv"


# Download the file
gdown.download(f"https://drive.google.com/uc?id={fileArr_id}", fileArr_name, quiet=False)

# Load the CSV file into a Pandas DataFrame
df_a_train = pd.read_csv(fileArr_name, header=9)

# Remove NaN entries
df_a_train = df_a_train.dropna()

df_a_train.set_index(df_a_train.columns[0], inplace=True)

# Transpose the DataFrame (convert months → columns, countries → rows)
df_a_train = df_a_train.T.reset_index()

# Rename columns
df_a_train.rename(columns={"index": "Country"}, inplace=True)

# Reshape using melt()
df_arr_train = df_a_train.melt(id_vars=["Country"], var_name="Month (YYYY-MM)", value_name="Arrivals")

# Save or display results
print(df_arr_train.head())  # View first few rows


In [ ]:

pd.set_option('display.max_colwidth', None)  # Show full content of each column
pd.set_option('display.max_rows', None)      # Show all rows
pd.set_option('display.max_columns', None)

# Remove the common prefix and suffix from the 'Country' column
df_arr_train['Country'] = df_arr_train['Country'].str.replace('Number Of Air Passenger Arrivals -> ', '', regex=False)
df_arr_train['Country'] = df_arr_train['Country'].str.replace(' (Number)', '', regex=False)

# Display the updated DataFrame
print(df_arr_train.head(30))

In [ ]:
# Create a new DataFrame with only country names
df_countries_arr_train = df_arr_train[df_arr_train['Country'].str.contains('->')].copy()

# Extract the country names by splitting and taking the last part
df_countries_arr_train['Country'] = df_countries_arr_train['Country'].apply(lambda x: x.split('->')[-1].strip())

# Rest index numbering for each entry
df_countries_index_arr_train = df_countries_arr_train.reset_index(drop=True)

# Change to interger type for Arrivals column
df_countries_index_arr_train['Arrivals'] = pd.to_numeric(df_countries_index_arr_train['Arrivals'], errors='coerce').fillna(0).astype(int)

# Changing format for date 
df_countries_index_arr_train['Month (YYYY-MM)'] = pd.to_datetime(
    df_countries_index_arr_train['Month (YYYY-MM)'].str.strip(), format='%Y %b'
).dt.strftime('%Y-%m')

# Export to CSV
df_countries_index_arr_train.to_csv("arrivals_train_final.csv", index=False)

# Display the new DataFrame
df_countries_index_arr_train

## Section 1.3 : Merging Flight Departures & Arrivals For Testing

In [21]:
import pandas as pd

# Merge based on 'Country' and 'Month (YYYY-MM)'
merged_df_train = pd.merge(df_countries_index_arr_train, df_countries_index_dep_train, on=["Country", "Month (YYYY-MM)"], how="inner")

merged_df_train.to_csv("merged_flight_train.csv",index=False )
